# PySpark

## Cómo crear una sesión en Spark

In [ ]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd

spark = SparkSession.builder.master("local").appName("PySpark").config('spark.ui.port', '4050').getOrCreate()
spark

## Leer un csv 

In [ ]:
df_titanic = spark.read.csv('titanic.csv',header=True)

df_titanic.printSchema()

## Eliminando las columnas no necesarias

In [ ]:
NUMERIC_COLUMNS = ["Age","SibSp","Parch","Fare"]
ENUM_COLUMNS = ["Embarked","Pclass"]
LABEL_COLUMN = ["Survived"]

df_titanic = df_titanic.select(NUMERIC_COLUMNS + ENUM_COLUMNS + LABEL_COLUMN)
df_titanic.printSchema()

## Convirtiendo las columnas numéricas a flotante

In [ ]:
from pyspark.sql.functions import col
def cast_to_float(input_df):
    return input_df.select([col(col_name).cast("float") if col_name in NUMERIC_COLUMNS 
                            else col(col_name)
                            for col_name in input_df.columns])

df_titanic = df_titanic.transform(cast_to_float)
df_titanic.printSchema()

In [ ]:
df_titanic.show()   

In [ ]:
df_titanic.select(["Age","SibSp","Parch","Pclass"]).summary().show()